# Adding Trading Indicators for more data

In [1]:
import pandas as pd
import numpy as np
import ta

df = pd.read_csv('data/btc_usdt_futures_data.csv')

df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df = df.astype(float)
df

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2021-01-01 00:00:00,28948.19,29045.93,28706.00,28786.75,3181.960,1.609460e+12,9.183460e+07,24166.0,1277.389,3.689901e+07,0.0
2021-01-01 00:15:00,28786.92,28902.68,28752.30,28859.28,2215.315,1.609461e+12,6.386520e+07,17044.0,1209.372,3.486977e+07,0.0
2021-01-01 00:30:00,28860.00,28968.49,28859.99,28947.61,1153.678,1.609462e+12,3.336960e+07,12208.0,620.757,1.795427e+07,0.0
2021-01-01 00:45:00,28947.61,29055.00,28910.94,29015.00,1486.635,1.609463e+12,4.309520e+07,12524.0,876.324,2.540813e+07,0.0
2021-01-01 01:00:00,29015.00,29499.00,28975.46,29446.35,7476.067,1.609464e+12,2.187655e+08,45858.0,4619.428,1.352136e+08,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 23:00:00,66011.10,66011.10,65907.00,65912.70,837.445,1.718407e+12,5.522476e+07,8892.0,294.184,1.939923e+07,0.0
2024-06-14 23:15:00,65912.60,66062.50,65912.00,65994.10,606.700,1.718408e+12,4.003054e+07,9404.0,351.279,2.317525e+07,0.0
2024-06-14 23:30:00,65994.20,66062.00,65994.10,66013.70,426.894,1.718409e+12,2.818703e+07,7006.0,238.916,1.577588e+07,0.0


## BTC Historical Volatility

In [2]:
# Calculate log returns
df['log_return'] = np.log(df['close'] / df['close'].shift(1))

# Calculate rolling standard deviation of log returns to get realized volatility
window_size = 96  # 96 K bars window for 1 day realized volatility
df['realized_volatility'] = df['log_return'].rolling(window=window_size).std()

# Annualize the volatility
df['hv'] = df['realized_volatility'] * np.sqrt(365 * 24)

## Garman-Klass Volatility

In [3]:
# Calculate Garman-Klass Volatility
def garman_klass_volatility(high, low, open, close, window=10):
    log_hl = np.log(high / low)
    log_co = np.log(close / open)
    rs = 0.5 * log_hl ** 2 - (2 * np.log(2) - 1) * log_co ** 2
    return np.sqrt(rs.rolling(window=window).sum())

In [4]:
df['gkv'] = garman_klass_volatility(df['high'], df['low'], df['open'], df['close'])

## Relative Strength Index (RSI)

In [5]:
# Calculate RSI
df['rsi'] = ta.momentum.RSIIndicator(df['close']).rsi()

## Bollinger Bands

In [6]:
# Calculate Bollinger Bands
bollinger = ta.volatility.BollingerBands(df['close'])
df['bb_high'] = bollinger.bollinger_hband()
df['bb_low'] = bollinger.bollinger_lband()

## Average True Range (ATR)

In [7]:
# Calculate ATR
df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()

## Moving Average Convergence Divergence (MACD)

In [8]:
# Calculate MACD
macd = ta.trend.MACD(df['close'])
df['MACD'] = macd.macd()
df['MACD_signal'] = macd.macd_signal()
df['MACD_diff'] = macd.macd_diff()

## Dollar Volume

In [9]:
# Calculate Dollar Volume
df['Dollar_volume'] = df['close'] * df['volume']

## Moving Average (MA)

In [10]:
# Calculate Moving Averages
df['ma'] = ta.trend.sma_indicator(df['close'], window=96)

## Exponential Moving Average (EMA)

In [11]:
df['ema'] = ta.trend.ema_indicator(df['close'], window=96)

## Stochastic Oscillator

In [12]:
# Calculate Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
df['stoch_k'] = stoch.stoch()
df['stoch_d'] = stoch.stoch_signal()

## Average Directional Index (ADX)

In [13]:
# Calculate Average Directional Index (ADX)
adx = ta.trend.ADXIndicator(df['high'], df['low'], df['close'])
df['adx'] = adx.adx()
df['adx_neg'] = adx.adx_neg()
df['adx_pos'] = adx.adx_pos()

## Commodity Channel Index (CCI)

In [14]:
# Calculate Commodity Channel Index (CCI)
df['cci'] = ta.trend.cci(df['high'], df['low'], df['close'], window=96)

## On-Balance Volume (OBV)

In [15]:
# Calculate On-Balance Volume (OBV)
df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])

## Chaikin Money Flow (CMF)

In [16]:
# Calculate Chaikin Money Flow (CMF)
df['cmf'] = ta.volume.chaikin_money_flow(df['high'], df['low'], df['close'], df['volume'])

## Williams %R

In [17]:
# Calculate Williams %R
df['williams'] = ta.momentum.WilliamsRIndicator(df['high'], df['low'], df['close']).williams_r()

##  Parabolic SAR

In [18]:
# Calculate Parabolic SAR
df['parabolic_sar'] = ta.trend.PSARIndicator(df['high'], df['low'], df['close']).psar()

## Volume Weighted Average Price (VWAP)

In [19]:
# Calculate Volume Weighted Average Price (VWAP)
df['vwap'] = ta.volume.volume_weighted_average_price(df['high'], df['low'], df['close'], df['volume'])

## Fibonacci Retracement

In [20]:
# Calculate Fibonacci retracement levels
def fibonacci_retracement(df, period=96):
    high = df['high'].rolling(window=period).max()
    low = df['low'].rolling(window=period).min()
    diff = high - low
    levels = {
        'fibonacci_0.236': high - diff * 0.236,
        'fibonacci_0.382': high - diff * 0.382,
        'fibonacci_0.5': high - diff * 0.5,
        'fibonacci_0.618': high - diff * 0.618,
        'fibonacci_0.764': high - diff * 0.764
    }
    return pd.DataFrame(levels)

In [21]:
# Add Fibonacci levels to DataFrame
fibonacci_levels = fibonacci_retracement(df)
df = pd.concat([df, fibonacci_levels], axis=1)

## Save to CSV

In [22]:
# Select columns to exclude the first 11 columns
selected_columns_df = df.iloc[:, 13:]

selected_columns_df

,hv,gkv,rsi,bb_high,bb_low,atr,MACD,MACD_signal,MACD_diff,Dollar_volume,...,obv,cmf,williams,parabolic_sar,vwap,fibonacci_0.236,fibonacci_0.382,fibonacci_0.5,fibonacci_0.618,fibonacci_0.764
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,9.159829e+07,...,3181.960,NaN,NaN,28786.750000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,6.393240e+07,...,5397.275,NaN,NaN,28859.280000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,3.339622e+07,...,6550.953,NaN,NaN,29045.930000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,4.313471e+07,...,8037.588,NaN,NaN,28859.990000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,2.201429e+08,...,15513.655,NaN,NaN,28859.990000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 23:00:00,0.208511,0.009088,49.422100,66313.855609,65218.934391,246.846477,1.668331,-59.094284,60.762614,5.519826e+07,...,-3576630.161,0.043428,-58.592693,66619.796960,66017.151557,66820.3724,66482.5138,66209.45,65936.3862,65598.5276
2024-06-14 23:15:00,0.207695,0.009151,51.925152,66306.179403,65300.210597,239.964586,6.954238,-45.884579,52.838817,4.003862e+07,...,-3576023.461,-0.001057,-55.615019,66591.285082,66040.324536,66820.3724,66482.5138,66209.45,65936.3862,65598.5276
2024-06-14 23:30:00,0.207660,0.009093,52.534276,66314.835130,65347.924870,227.674258,12.579900,-34.191684,46.771583,2.818085e+07,...,-3575596.567,-0.054569,-60.624284,66563.913678,66059.796280,66820.3724,66482.5138,66209.45,65936.3862,65598.5276


In [23]:
# Save the DataFrame to a CSV file
selected_columns_df.to_csv("data/indicators.csv")
print("Data saved to data/indicators.csv")

Data saved to data/indicators.csv
